# Visualize Initial Board FEN Positions

This notebook visualizes all initial board FEN positions from the config file and displays their quality values to verify correctness.


In [1]:
import chess
import chess.svg
from IPython.display import SVG, HTML, display
from omegaconf import OmegaConf
import yaml
from pathlib import Path
from typing import Dict, Any
import sys
import os

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == 'test' else Path.cwd()
sys.path.insert(0, str(project_root))


In [2]:
# Load config file
config_path = project_root / "config" / "train_mcts.yaml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Extract initial_board_fen dictionary
initial_board_fen = config.get('training', {}).get('initial_board_fen', {})

print(f"Found {len(initial_board_fen)} initial board positions")
print(f"\nConfig structure:")
for i, (fen, value) in enumerate(initial_board_fen.items(), 1):
    if isinstance(value, dict):
        print(f"{i}. Weight: {value.get('weight', 'N/A')}, Quality: {value.get('quality', 'N/A')}")
    else:
        print(f"{i}. Weight: {value} (legacy format)")


Found 21 initial board positions

Config structure:
1. Weight: 0.5, Quality: equal
2. Weight: 0.025, Quality: winning
3. Weight: 0.025, Quality: winning
4. Weight: 0.025, Quality: winning
5. Weight: 0.025, Quality: winning
6. Weight: 0.025, Quality: equal
7. Weight: 0.025, Quality: losing
8. Weight: 0.025, Quality: winning
9. Weight: 0.025, Quality: equal
10. Weight: 0.025, Quality: losing
11. Weight: 0.025, Quality: equal
12. Weight: 0.025, Quality: losing
13. Weight: 0.025, Quality: winning
14. Weight: 0.025, Quality: winning
15. Weight: 0.025, Quality: equal
16. Weight: 0.025, Quality: winning
17. Weight: 0.025, Quality: winning
18. Weight: 0.025, Quality: winning
19. Weight: 0.025, Quality: winning
20. Weight: 0.025, Quality: winning
21. Weight: 0.025, Quality: equal


In [3]:
def parse_fen_info(fen: str) -> Dict[str, Any]:
    """Parse FEN string to extract board state and turn information."""
    parts = fen.split()
    board_part = parts[0] if len(parts) > 0 else fen
    turn = parts[1] if len(parts) > 1 else 'w'
    
    # Count pieces
    piece_count = sum(1 for char in board_part if char.isalpha())
    
    return {
        'board_part': board_part,
        'turn': turn,
        'turn_color': 'White' if turn == 'w' else 'Black',
        'piece_count': piece_count,
        'full_fen': fen
    }

def get_quality_color(quality: str) -> str:
    """Get color for quality label."""
    colors = {
        'winning': '#28a745',  # Green
        'equal': '#ffc107',    # Yellow/Orange
        'losing': '#dc3545'     # Red
    }
    return colors.get(quality, '#6c757d')  # Gray for unknown


In [4]:
def visualize_position(fen: str, weight: float, quality: str = None, index: int = None, description: str = None):
    """Visualize a single chess position with its quality."""
    try:
        board = chess.Board(fen)
        fen_info = parse_fen_info(fen)
        
        # Create SVG
        svg = chess.svg.board(
            board=board,
            size=400,
            orientation=chess.WHITE  # Always show from white's perspective
        )
        
        # Build info HTML with better contrast
        quality_html = ''
        if quality:
            quality_html = f'<p><strong style="color: #212529;">Quality:</strong> <span style="color: {get_quality_color(quality)}; font-weight: bold; font-size: 1.2em;">{quality.upper()}</span> <span style="color: #495057;">(from white\'s perspective)</span></p>'
        else:
            quality_html = '<p><strong style="color: #212529;">Quality:</strong> <span style="color: #495057;">Not specified</span></p>'
        
        description_html = f'<p style="color: #495057;"><em>{description}</em></p>' if description else ''
        
        info_html = f"""
        <div style="margin: 10px; padding: 10px; border: 2px solid #ddd; border-radius: 5px; background-color: #f8f9fa; min-width: 300px; color: #212529;">
            <h3 style="margin-top: 0; color: #212529;">Position #{index if index else 'N/A'}</h3>
            {description_html}
            <p style="color: #212529;"><strong>Weight:</strong> <span style="color: #495057;">{weight:.3f} ({weight*100:.1f}%)</span></p>
            {quality_html}
            <p style="color: #212529;"><strong>Turn:</strong> <span style="color: #495057;">{fen_info['turn_color']} to move</span></p>
            <p style="color: #212529;"><strong>Piece Count:</strong> <span style="color: #495057;">{fen_info['piece_count']} pieces</span></p>
            <p style="color: #212529;"><strong>FEN:</strong> <code style="font-size: 0.9em; word-break: break-all; color: #212529; background-color: #e9ecef; padding: 2px 4px; border-radius: 3px;">{fen}</code></p>
        </div>
        """
        
        # Display
        display(HTML(f"<div style='display: flex; gap: 20px; margin: 20px 0;'>{info_html}<div>{SVG(svg).data}</div></div>"))
        
    except Exception as e:
        print(f"Error visualizing position {index}: {e}")
        print(f"FEN: {fen}")


## All Initial Board Positions


In [5]:
# Visualize all positions
for index, (fen, value) in enumerate(initial_board_fen.items(), 1):
    # Handle both legacy format (float) and new format (dict)
    if isinstance(value, dict):
        weight = value.get('weight', 0.0)
        quality = value.get('quality', None)
    else:
        weight = float(value)
        quality = None
    
    # Extract description from config comments (if available)
    description = None  # Could be extracted from YAML comments if needed
    
    visualize_position(fen, weight, quality, index, description)
    print("\n" + "="*80 + "\n")


## Summary Statistics


In [6]:
# Calculate statistics
total_weight = 0
quality_counts = {'winning': 0, 'equal': 0, 'losing': 0, 'unknown': 0}
turn_counts = {'White': 0, 'Black': 0}

for fen, value in initial_board_fen.items():
    if isinstance(value, dict):
        weight = value.get('weight', 0.0)
        quality = value.get('quality', None)
    else:
        weight = float(value)
        quality = None
    
    total_weight += weight
    
    if quality:
        quality_counts[quality] = quality_counts.get(quality, 0) + 1
    else:
        quality_counts['unknown'] += 1
    
    fen_info = parse_fen_info(fen)
    turn_counts[fen_info['turn_color']] += 1

print(f"Total positions: {len(initial_board_fen)}")
print(f"Total weight: {total_weight:.3f} (should be ~1.0)")
print(f"\nQuality distribution:")
for quality, count in quality_counts.items():
    if count > 0:
        print(f"  {quality.capitalize()}: {count}")
print(f"\nTurn distribution:")
for turn, count in turn_counts.items():
    print(f"  {turn}: {count}")


Total positions: 21
Total weight: 1.000 (should be ~1.0)

Quality distribution:
  Winning: 12
  Equal: 6
  Losing: 3

Turn distribution:
  White: 18
  Black: 3


## Quality Verification

Check if quality values make sense based on material and position:


In [7]:
def analyze_position_material(fen: str) -> Dict[str, Any]:
    """Analyze material balance in a position."""
    board = chess.Board(fen)
    
    # Count material
    white_material = {
        'Q': len(board.pieces(chess.QUEEN, chess.WHITE)),
        'R': len(board.pieces(chess.ROOK, chess.WHITE)),
        'B': len(board.pieces(chess.BISHOP, chess.WHITE)),
        'N': len(board.pieces(chess.KNIGHT, chess.WHITE)),
        'P': len(board.pieces(chess.PAWN, chess.WHITE))
    }
    
    black_material = {
        'Q': len(board.pieces(chess.QUEEN, chess.BLACK)),
        'R': len(board.pieces(chess.ROOK, chess.BLACK)),
        'B': len(board.pieces(chess.BISHOP, chess.BLACK)),
        'N': len(board.pieces(chess.KNIGHT, chess.BLACK)),
        'P': len(board.pieces(chess.PAWN, chess.BLACK))
    }
    
    # Simple material count (approximate)
    piece_values = {'Q': 9, 'R': 5, 'B': 3, 'N': 3, 'P': 1}
    white_value = sum(white_material[p] * piece_values[p] for p in piece_values)
    black_value = sum(black_material[p] * piece_values[p] for p in piece_values)
    
    material_diff = white_value - black_value
    
    return {
        'white_material': white_material,
        'black_material': black_material,
        'white_value': white_value,
        'black_value': black_value,
        'material_diff': material_diff,
        'is_checkmate': board.is_checkmate(),
        'is_stalemate': board.is_stalemate(),
        'is_insufficient_material': board.is_insufficient_material()
    }

# Analyze each position
print("Position Analysis:\n")
print("="*100)

for index, (fen, value) in enumerate(initial_board_fen.items(), 1):
    if isinstance(value, dict):
        quality = value.get('quality', None)
    else:
        quality = None
    
    analysis = analyze_position_material(fen)
    fen_info = parse_fen_info(fen)
    
    print(f"\nPosition #{index}:")
    print(f"  FEN: {fen}")
    print(f"  Quality (config): {quality if quality else 'Not specified'}")
    print(f"  Turn: {fen_info['turn_color']}")
    print(f"  Material difference (White - Black): {analysis['material_diff']:+d}")
    print(f"  White material: {analysis['white_material']}")
    print(f"  Black material: {analysis['black_material']}")
    
    # Check if quality matches material
    if quality:
        expected_quality = None
        if analysis['material_diff'] > 2:
            expected_quality = 'winning'
        elif analysis['material_diff'] < -2:
            expected_quality = 'losing'
        else:
            expected_quality = 'equal'
        
        # Note: Quality in config is from white's perspective, so we compare directly
        # (we don't flip based on turn since config quality is always from white's perspective)
        match = "✓" if quality == expected_quality else "✗"
        print(f"  Expected quality (from material): {expected_quality} {match}")
        if match == "✗":
            print(f"    ⚠️  Warning: Quality mismatch! Config says '{quality}' but material suggests '{expected_quality}'")
            print(f"    Note: This might be correct if position factors (not just material) affect the evaluation")
    
    if analysis['is_checkmate']:
        print(f"  ⚠️  Checkmate position!")
    if analysis['is_stalemate']:
        print(f"  ⚠️  Stalemate position!")
    if analysis['is_insufficient_material']:
        print(f"  ℹ️  Insufficient material (draw)")
    
    print("-"*100)


Position Analysis:


Position #1:
  FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
  Quality (config): equal
  Turn: White
  Material difference (White - Black): +0
  White material: {'Q': 1, 'R': 2, 'B': 2, 'N': 2, 'P': 8}
  Black material: {'Q': 1, 'R': 2, 'B': 2, 'N': 2, 'P': 8}
  Expected quality (from material): equal ✓
----------------------------------------------------------------------------------------------------

Position #2:
  FEN: 8/8/8/4k3/8/3K4/Q7/8 w - - 0 1
  Quality (config): winning
  Turn: White
  Material difference (White - Black): +9
  White material: {'Q': 1, 'R': 0, 'B': 0, 'N': 0, 'P': 0}
  Black material: {'Q': 0, 'R': 0, 'B': 0, 'N': 0, 'P': 0}
  Expected quality (from material): winning ✓
----------------------------------------------------------------------------------------------------

Position #3:
  FEN: 8/8/8/3k4/8/3K4/R7/8 w - - 0 1
  Quality (config): winning
  Turn: White
  Material difference (White - Black): +5
  White material: {'